# Analisador Léxico
Construir um analisador léxico para uma linguagem de query com a qual se podem escrever frases do género:

```sparql
# DBPedia: obras de Chuck Berry

select ?nome ?desc where { 
    ?s a dbo:MusicalArtist. 
    ?s foaf:name "Chuck Berry"@en . 
    ?w dbo:artist ?s. 
    ?w foaf:name ?nome. 
    ?w dbo:abstract ?desc 
} LIMIT 1000

In [25]:
import re

def analisador_lexico_personalizado(texte):
    # 1. Definição das Regras Léxicas
    tokens = [
        # Tokens de Controlo
        ('COMMENT', r'#.*'),
        ('SKIP', r'[ \t]+'),
        ('NEWLINE', r'\n'),
        
        # Palavras-Chave (Aceitam maiúsculas e minúsculas)
        ('SELECT', r'SELECT\b|select\b'),
        ('WHERE', r'WHERE\b|where\b'),
        ('LIMIT', r'LIMIT\b|limit\b'),
        ('PREFIX',r'PREFIX\b'),
        ('OPTIONAL',r'OPTIONAL\b'),
        ('FILTER',r'FILTER\b'),
        
        # Literais
        # Corrigida para strings (aceita a tag @en opcional)
        ('STRING', r'"(\\["\\]|[^"\\])*"(@[a-z]{2})?'),
        ('INT', r'\d+'),
        
        # Estruturas de Identificação (Crucial para o seu gabarito)
        # VAR: Não inclui o '?', fazendo-o ser tokenizado como 'ERROR'
        ('VAR', r'[a-zA-Z_][\w]*'),
        # IDENT: Nomes Prefixados, Locais e Simples
        ('IDENT', r'([a-zA-Z_][\w]*:[a-zA-Z_][\w]*)|:[a-zA-Z_][\w]*|[a-zA-Z_][\w]*'),
        
        # Símbolos
        ('OP',r'[=!<>]+'),
        # PUNCT: Engloba chavetas e símbolos
        ('PUNCT', r'[{}.;,]'),
        
        # Fallback
        ('ERROR', r'.')
    ]
    
    lista_tokens=[]
    linha=1
    
    # Construção da Regex Geral
    regex='|'.join(f'(?P<{name}>{pattern})' for name, pattern in tokens)
    val=re.finditer(regex, texte)

    for m in val:
        grupos_capturados=m.groupdict()
        tipot=None
        valort=m.group()
        
        for key in grupos_capturados:
            if grupos_capturados[key]:
                tipot=key
                break
        
        # Lógica de Controlo
        if tipot=='NEWLINE':
            linha+=1
            lista_tokens.append((tipot, valort, linha, m.span()))
            
        elif tipot != 'SKIP' and tipot != 'COMMENT':
            lista_tokens.append((tipot, valort, linha, m.span()))
            
    return lista_tokens

# 2. Teste e Impressão
# String de teste
texte="""SELECT ?ola ?jorge ?tudo WHERE 
{
  ?ola a :Pessoa ;
     :temIdade ?19 ;
     :eIrmaoDe ?jorge .
}
"""

resultado = analisador_lexico_personalizado(texte)

# Imprime a lista de tokens toda seguida, como um único output de lista.
print(resultado)

[('SELECT', 'SELECT', 1, (0, 6)), ('ERROR', '?', 1, (7, 8)), ('VAR', 'ola', 1, (8, 11)), ('ERROR', '?', 1, (12, 13)), ('VAR', 'jorge', 1, (13, 18)), ('ERROR', '?', 1, (19, 20)), ('VAR', 'tudo', 1, (20, 24)), ('WHERE', 'WHERE', 1, (25, 30)), ('NEWLINE', '\n', 2, (31, 32)), ('PUNCT', '{', 2, (32, 33)), ('NEWLINE', '\n', 3, (33, 34)), ('ERROR', '?', 3, (36, 37)), ('VAR', 'ola', 3, (37, 40)), ('VAR', 'a', 3, (41, 42)), ('IDENT', ':Pessoa', 3, (43, 50)), ('PUNCT', ';', 3, (51, 52)), ('NEWLINE', '\n', 4, (52, 53)), ('IDENT', ':temIdade', 4, (58, 67)), ('ERROR', '?', 4, (68, 69)), ('INT', '19', 4, (69, 71)), ('PUNCT', ';', 4, (72, 73)), ('NEWLINE', '\n', 5, (73, 74)), ('IDENT', ':eIrmaoDe', 5, (79, 88)), ('ERROR', '?', 5, (89, 90)), ('VAR', 'jorge', 5, (90, 95)), ('PUNCT', '.', 5, (96, 97)), ('NEWLINE', '\n', 6, (97, 98)), ('PUNCT', '}', 6, (98, 99)), ('NEWLINE', '\n', 7, (99, 100))]
